In [1]:
from matplotlib import pyplot
import os
import numpy as np
import pickle
import cv2
from PIL import Image
from numpy import asarray, expand_dims
from keras.models import load_model
from keras_facenet import FaceNet
from os import listdir

In [2]:
myfile = open("data.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [14]:
HaarCascade = cv2.CascadeClassifier('model/haarcascade_frontalface_default.xml')
MyFaceNet = FaceNet()

In [15]:
print(HaarCascade)

< cv2.CascadeClassifier 0000024EA65517D0>


In [ ]:
cap = cv2.VideoCapture(0)

while(1):
    _, gbr1 = cap.read()
    
    wajah = HaarCascade.detectMultiScale(gbr1,1.1,4)
    
    if len(wajah)>0:
        x1, y1, width, height = wajah[0]        
    else:
        x1, y1, width, height = 1, 1, 10, 10
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)                  # konversi dari OpenCV ke PIL
    gbr_array = asarray(gbr)
    
    face = gbr_array[y1:y2, x1:x2]                        
    
    face = Image.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)
    
#     face = face.astype('float32')
#     mean, std = face.mean(), face.std()
#     face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
#     signature = MyFaceNet.predict(face)
    signature = MyFaceNet.embeddings(face)
    
    min_dist=100
    identity=' '
    for key, value in database.items() :
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
            
    cv2.putText(gbr1,identity, (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.rectangle(gbr1,(x1,y1),(x2,y2), (0,255,0), 2)
        
    cv2.imshow('res',gbr1)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()

In [13]:
import cv2
from PIL import Image
from numpy import asarray, expand_dims
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Your model initialization and database code here

cap = cv2.VideoCapture(0)

while(1):
    _, gbr1 = cap.read()
    
    wajah = HaarCascade.detectMultiScale(gbr1, 1.1, 4)
    
    if len(wajah) > 0:
        x1, y1, width, height = wajah[0]        
    else:
        x1, y1, width, height = 1, 1, 10, 10
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)                  
    gbr_array = asarray(gbr)
    
    face = gbr_array[y1:y2, x1:x2]                        
    face = Image.fromarray(face)                       
    face = face.resize((160, 160))
    face = asarray(face)
    face = expand_dims(face, axis=0)

    signature = MyFaceNet.embeddings(face)
    
    max_similarity = -1
    identity = ' '
    uk = 'Unknown'
    for key, value in database.items():
        similarity = cosine_similarity(value.reshape(1, -1), signature.reshape(1, -1))[0][0]
        if similarity > max_similarity:
            max_similarity = similarity
            identity = key
    
    confidence = max_similarity * 100  # Calculate confidence in percentage
    
    if confidence < 50:
        cv2.putText(gbr1, f'{uk} ({confidence:.2f}%)', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    else:
        cv2.putText(gbr1, f'{identity} ({confidence:.2f}%)', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.rectangle(gbr1, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
    cv2.imshow('res', gbr1)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()


1/1 [==============================] - 0s 260ms/step


1/1 [==============================] - 0s 266ms/step
